In [19]:
import pandas as pd 
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, RepeatedStratifiedKFold
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score
from sklearn.preprocessing import MinMaxScaler
import glob
import os
import time 

In [20]:
train_path = max(glob.glob('/Users/ivankoh/Library/CloudStorage/OneDrive-NationalUniversityofSingapore/1D/NUS Y3S1/BT4222/Data/train/*.csv'), key=os.path.getctime) 
print(train_path)
df = pd.read_csv(train_path)

/Users/ivankoh/Library/CloudStorage/OneDrive-NationalUniversityofSingapore/1D/NUS Y3S1/BT4222/Data/train/kickstarter_train_final_20221108-132028.csv


In [21]:
df 

,reward_tiers,min_reward,max_reward,goal,state,staff_pick,has_video,rewards_0,rewards_1,rewards_2,...,x0_UY,x0_VE,x0_VN,x0_VU,x0_WS,x0_XK,x0_YE,x0_ZA,x0_ZW,x0_nan
0,0.041667,0.000167,0.004995,0.000090,1,1,1,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.058333,0.000067,0.004495,0.000050,1,1,1,0.069896,0.069940,0.101049,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.058333,0.000033,0.000525,0.000006,1,0,1,0.077258,0.077307,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.041667,0.000133,0.010990,0.000340,1,0,1,0.014734,0.014744,0.021302,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.066667,0.000007,0.004995,0.000200,1,1,1,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24751,0.233333,0.000007,0.002495,0.000015,1,0,0,0.101029,0.094775,0.118673,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
24752,0.100000,0.000033,0.004995,0.000022,1,0,1,0.034785,0.034807,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
24753,0.083333,0.000133,0.000345,0.000005,1,0,0,0.026888,0.026905,0.019436,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
24754,0.075000,0.000067,0.024995,0.000150,1,1,1,0.034953,0.034975,0.016844,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Get to the correct data format

In [22]:
X_train, y_train = df.drop('state', axis=1).to_numpy(), df['state'].to_numpy()

## Randomized Search with K-fold CV applied to SVC

**Hyperparameter Tuning**

We will use RandomizedSearchCV for hyperparameter tuning.

### 1. Training

In [23]:
# Hyper-parameters
gamma = [1e-7,1e-4,1e-3,1e-1,1e0,1e1]
C = [1e-3,1e-2,1e0,1e1,1e3,1e5]
kernels = ['rbf', 'linear']  #poly takes forever to train 'sigmoid'

params = {'C': C, 'gamma': gamma, 'kernel': kernels}
cv_method = RepeatedStratifiedKFold(n_splits=5, 
                                    n_repeats=3, 
                                    random_state=2022)
svc_randomcv = RandomizedSearchCV(
    estimator=SVC(max_iter=500),
    param_distributions=params,
    n_iter=10, 
    cv=cv_method,
    verbose=2,
    random_state=2022,
    # n_jobs=-1, # use all processors,
    scoring='roc_auc'
)

svc_randomcv.fit(X_train, y_train)

Fitting 15 folds for each of 10 candidates, totalling 150 fits


/Users/ivankoh/opt/miniconda3/envs/bt4222/lib/python3.9/site-packages/sklearn/svm/_base.py:301: ConvergenceWarning: Solver terminated early (max_iter=500).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(


[CV] END ..................C=100000.0, gamma=1.0, kernel=rbf; total time=   5.2s


/Users/ivankoh/opt/miniconda3/envs/bt4222/lib/python3.9/site-packages/sklearn/svm/_base.py:301: ConvergenceWarning: Solver terminated early (max_iter=500).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(


[CV] END ..................C=100000.0, gamma=1.0, kernel=rbf; total time=   5.2s


...

[CV] END ...............C=0.001, gamma=0.0001, kernel=linear; total time=   4.8s


/Users/ivankoh/opt/miniconda3/envs/bt4222/lib/python3.9/site-packages/sklearn/svm/_base.py:301: ConvergenceWarning: Solver terminated early (max_iter=500).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(


[CV] END ...............C=0.001, gamma=0.0001, kernel=linear; total time=   4.9s


/Users/ivankoh/opt/miniconda3/envs/bt4222/lib/python3.9/site-packages/sklearn/svm/_base.py:301: ConvergenceWarning: Solver terminated early (max_iter=500).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(


RandomizedSearchCV(cv=RepeatedStratifiedKFold(n_repeats=3, n_splits=5, random_state=2022),
                   estimator=SVC(max_iter=500),
                   param_distributions={'C': [0.001, 0.01, 1.0, 10.0, 1000.0,
                                              100000.0],
                                        'gamma': [1e-07, 0.0001, 0.001, 0.1,
                                                  1.0, 10.0],
                                        'kernel': ['rbf', 'linear']},
                   random_state=2022, scoring='roc_auc', verbose=2)

### 2. Best Hyper-parameters

In [24]:
print(svc_randomcv.best_params_)
print(svc_randomcv.best_score_)

{'kernel': 'rbf', 'gamma': 1e-07, 'C': 10.0}
0.8831258276768165


### 3. Testing

In [25]:
test_path = max(glob.glob('/Users/ivankoh/Library/CloudStorage/OneDrive-NationalUniversityofSingapore/1D/NUS Y3S1/BT4222/Data/test/*.csv'), key=os.path.getctime) 
print(test_path)
test = pd.read_csv(test_path)

X_test, y_test = test.drop('state', axis=1).to_numpy(), test['state'].to_numpy()

/Users/ivankoh/Library/CloudStorage/OneDrive-NationalUniversityofSingapore/1D/NUS Y3S1/BT4222/Data/test/kickstarter_test_final_20221108-132033.csv


In [26]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(24756, 578)
(24756,)
(4369, 578)
(4369,)


In [27]:
best_clf = SVC(**svc_randomcv.best_params_)

start_time = time.time()
best_clf.fit(X_train, y_train)
y_pred_test = best_clf.predict(X_test)

time_taken = time.time() - start_time
print("Total time taken for the program execution", time_taken) # seconds

print(classification_report(y_test, y_pred_test, digits=5))
roc_auc_score(y_test, y_pred_test)

Total time taken for the program execution 109.81598997116089
              precision    recall  f1-score   support

           0    0.00000   0.00000   0.00000      1282
           1    0.70657   1.00000   0.82806      3087

    accuracy                        0.70657      4369
   macro avg    0.35328   0.50000   0.41403      4369
weighted avg    0.49924   0.70657   0.58508      4369



/Users/ivankoh/opt/miniconda3/envs/bt4222/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/ivankoh/opt/miniconda3/envs/bt4222/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/ivankoh/opt/miniconda3/envs/bt4222/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier

0.5